<a> <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTTYOPfdkntp9QtXcPkvO7JsGnU9GunyWgQ8TvyHBySz38_j0UAYQ" align="center" ></a>

<h1 align="center"> Segmentation and Clustering Neighborhoods in Ontario(CAN)</h1>